In [1]:
OPINION_FILE = "../scripts/data/plastic_bags_so_opinions.txt"
RESPONSE_FILE = "../data/raw/plastic_bags_so.txt"

OPINION_FILE = "../scripts/data/eating_meat_so_opinions.txt"
RESPONSE_FILE = "../data/raw/methane_so.tsv"

with open(OPINION_FILE) as i:
    opinionated_responses = set([line.strip() for line in i])
    
with open(RESPONSE_FILE) as i:
    all_responses = set([line.strip() for line in i])

objective_responses = set([r for r in all_responses if r not in opinionated_responses])
    
print("Opinionated:", len(opinionated_responses))
print("Non-opinionated:", len(objective_responses))
print("All:", len(all_responses))

Opinionated: 550
Non-opinionated: 286
All: 836


In [2]:
from sklearn.model_selection import train_test_split

text_and_labels = []

for text in opinionated_responses:
    text_and_labels.append((text, "opinionated"))

for text in objective_responses:
    text_and_labels.append((text, "objective"))
    
texts, labels = zip(*text_and_labels)

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.3, random_state=1)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

preprocessing = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer())
])
  
print("Preprocessing training data...")
train_preprocessed = preprocessing.fit_transform(train_texts)

print("Preprocessing test data...")
test_preprocessed = preprocessing.transform(test_texts)

Preprocessing training data...
Preprocessing test data...


In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

nb_classifier = MultinomialNB()
svm_classifier = LinearSVC()
lr_classifier = LogisticRegression(multi_class="ovr")

print("Training Naive Bayes classifier...")
nb_classifier.fit(train_preprocessed, train_labels)

print("Training SVM classifier...")
svm_classifier.fit(train_preprocessed, train_labels)

print("Training Logistic Regression classifier...")
lr_classifier.fit(train_preprocessed, train_labels)

Training Naive Bayes classifier...
Training SVM classifier...
Training Logistic Regression classifier...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [5]:
nb_predictions = nb_classifier.predict(test_preprocessed)
svm_predictions = svm_classifier.predict(test_preprocessed)
lr_predictions = lr_classifier.predict(test_preprocessed)

In [6]:
import numpy as np

print("NB Accuracy:", np.mean(nb_predictions == test_labels))
print("SVM Accuracy:", np.mean(svm_predictions == test_labels))
print("LR Accuracy:", np.mean(lr_predictions == test_labels))

NB Accuracy: 0.8087649402390438
SVM Accuracy: 0.8924302788844621
LR Accuracy: 0.8565737051792829


In [7]:
import eli5

eli5.explain_weights(svm_classifier, 
                     feature_names = preprocessing.named_steps["vect"].get_feature_names(),
                     target_names = ["opinionated", "objective"]
                    )

/Users/yvespeirsman/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/yvespeirsman/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Explanation(estimator="LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,\n          intercept_scaling=1, loss='squared_hinge', max_iter=1000,\n          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,\n          verbose=0)", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='objective', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='we', weight=2.406453491933625, std=None, value=None), FeatureWeight(feature='should', weight=2.2223678649004777, std=None, value=None), FeatureWeight(feature='need', weight=1.2253556631260387, std=None, value=None), FeatureWeight(feature='could', weight=1.2217310010771372, std=None, value=None), FeatureWeight(feature='reduces', weight=1.1786174479520517, std=None, value=None), FeatureWeight(feature='reducing', weight=1.0791416356825523, std=None, value=None), FeatureWeight(feature='find', weight=1.0307337262605212, std=None, value=None)], neg=[FeatureWeight(feature='scientists', weight=-1.5559176325580641, std=None, value=None), FeatureWeight(feature='impossible', weight=-1.4017124229869828, std=None, value=None), FeatureWeight(feature='wants', weight=-1.3984863239918364, std=None, value=None), FeatureWeight(feature='created', weight=-1.3082936856764606, std=None, value=None), FeatureWeight(feature='want', weight=-1.2787555975984988, std=None, value=None), FeatureWeight(feature='some', weight=-1.1110149743825555, std=None, value=None), FeatureWeight(feature='thinks', weight=-1.099373717013908, std=None, value=None), FeatureWeight(feature='many', weight=-1.0958179966270574, std=None, value=None), FeatureWeight(feature='encouraged', weight=-1.094493274156262, std=None, value=None), FeatureWeight(feature='companies', weight=-1.0636559326125985, std=None, value=None), FeatureWeight(feature='are', weight=-1.0395065395101832, std=None, value=None), FeatureWeight(feature='have', weight=-1.0357587249994138, std=None, value=None), FeatureWeight(feature='argue', weight=-0.9839732072943537, std=None, value=None)], pos_remaining=418, neg_remaining=304), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [8]:
import spacy

def simple_test(tokens):
    return "should" in tokens or "need" in tokens or "must" in tokens or "needs" in tokens or "important" in tokens or "we" in tokens or "find" in tokens
    

def test_simple_classifier(texts, labels):
    
    nlp = spacy.load("en")
    predicted_labels = []
    correct = 0
    
    for text, label in zip(texts, labels):
        tokens = set([t.text.lower() for t in nlp(text)])
        if simple_test(tokens) and label == "opinionated":
            correct += 1
        elif not simple_test(tokens) and label == "objective":
            correct += 1
            
    return correct/len(texts)

print(test_simple_classifier(texts, labels))

0.7990430622009569
